<a href="https://colab.research.google.com/github/FaiazS/Knowledge_Distillation_LLM-Distillation-Step-by-Step-/blob/main/Knowledge_Distillation_LLM(Distilling_Step_by_Step).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch datasets

In [2]:
import torch

from transformers import AutoTokenizer

from transformers import AutoModelForMaskedLM

from transformers import Trainer

from transformers import TrainingArguments

from datasets import load_dataset

In [3]:
student_model = "google-bert/bert-base-uncased"

from transformers import AutoModelForSequenceClassification

student_llm_model = AutoModelForSequenceClassification.from_pretrained(student_model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
print(student_llm_model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [5]:
dataset = load_dataset("sst2",split = "train[:10]")

In [6]:
print(dataset)

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 10
})


In [7]:
print(dataset[0])

{'idx': 0, 'sentence': 'hide new secretions from the parental units ', 'label': 0}


In [8]:
!pip install ollama

!pip install langchain-ollama

In [9]:
#from langchain_ollama import ChatOllama

#llm_chat_engine = ChatOllama(
                             #model="deepseek-r1:7b",
                             #base_url="127.0.0.1:11434",
                             #temperature=0.3
#)

#test_prompt = "I am very joyful today"

#student_llm_response = llm_chat_engine.invoke(test_prompt)

#print(student_llm_response.content)

In [10]:
!pip install langchain

!pip install langchain-groq

In [11]:
from google.colab import userdata

import os

from langchain_groq import ChatGroq

os.environ['GROQ_API_KEY'] = userdata.get('LLMProjectKey1')

teacher_model = ChatGroq(

                         model = "deepseek-r1-distill-llama-70b",

                         temperature = 0.3

)

In [12]:
test_prompt = "I am very joyous today"

teacher_model_response = teacher_model.invoke(test_prompt)

print(teacher_model_response.content)

<think>
Okay, so I'm trying to figure out how to respond to the user's message. They said, "I am very joyous today." Hmm, that's a positive statement. I should probably acknowledge their feelings and maybe ask a follow-up question to keep the conversation going.

First, I want to express that I'm glad they're feeling joyous. Maybe start with something like, "That's wonderful to hear!" or "I'm so glad you're feeling joyous today!" Then, I should ask an open-ended question to encourage them to share more. Perhaps, "What's making you feel so joyful today?" or "What's bringing you joy at the moment?"

Wait, I should make sure my response is friendly and supportive. Maybe add an emoji to convey warmth, like a smiley face or a heart. But I'm not sure if emojis are appropriate here. The user didn't use any, so maybe it's better to keep it professional.

Alternatively, I could say something like, "That's great! Would you like to share what's making you happy today?" This way, I'm inviting them

In [13]:
def generate_model_reasoning(input_text):

  """
  Uses Groq's Deepseek R1 Model to generate a step by step Rationale(Reasoning) for the Output given for particular Input(User prompt)
  """

  user_prompt = f"Deliver step by step reasoning before responding to: {input_text}" #Chain of thoughts prompting request for Step by Step Distillation

  teacher_model_response = teacher_model.invoke(user_prompt)

  return teacher_model_response.content if hasattr(teacher_model_response, "content") else teacher_model_response

teacher_model_response = generate_model_reasoning("Explain AI Agents and the technologies used behind the same, keep in mind it should be widely adopted in Industry")

In [14]:
print(teacher_model_response)

<think>
Okay, so I need to explain AI agents and the technologies behind them, especially focusing on how they're widely adopted in industries. Hmm, where do I start? I remember AI agents are like autonomous entities that can perform tasks on their own, but I'm not entirely sure. Maybe I should break it down.

First, I think I should define what an AI agent is. From what I've heard, an AI agent is a program that uses AI to make decisions and act on its own. It interacts with an environment, right? So, it's not just a tool that does one thing; it's more like a system that can adapt and learn.

Next, I need to outline the key characteristics. I remember something about autonomy, meaning they can operate without human intervention. Then there's reactivity, so they can respond to changes. Proactivity might be another trait, where they take initiative. Also, social ability, interacting with other agents or humans. Learning and adaptation are probably important too, as they improve over time

In [15]:
from transformers import AutoTokenizer

llm_tokenizer = AutoTokenizer.from_pretrained(student_model)

In [16]:
#Prepare datasets with Rationales(Reasoning)

def process_data(example):

  input_text = example["sentence"]

  reasoning = generate_model_reasoning(input_text)

  label = example["label"]

  #Tokenizing input and rationale/reasoning

  input_tokens = llm_tokenizer(input_text, truncation ="longest_first", padding = True, max_length = 256)

  reasoning_tokens = llm_tokenizer(reasoning, truncation = "longest_first", padding = True, max_length = 256)

  return {

          "input_ids" : input_tokens["input_ids"],

          "attention_mask" : input_tokens["attention_mask"],

          "labels" : label,

          "reasoning_ids " : reasoning_tokens["input_ids"],

          "reasoning_mask" : reasoning_tokens["attention_mask"]
  }

processed_dataset = dataset.map(process_data)

Parameter 'function'=<function process_data at 0x78b425ac13a0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [17]:
print(processed_dataset)

Dataset({
    features: ['idx', 'sentence', 'label', 'input_ids', 'attention_mask', 'labels', 'reasoning_ids ', 'reasoning_mask'],
    num_rows: 10
})


In [18]:
from datasets import Dataset

processed_dataset.save_to_disk('preprocessed_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

In [19]:
from datasets import load_from_disk

processed_dataset.load_from_disk('preprocessed_dataset')

Dataset({
    features: ['idx', 'sentence', 'label', 'input_ids', 'attention_mask', 'labels', 'reasoning_ids ', 'reasoning_mask'],
    num_rows: 10
})

In [20]:
#Training the Student Model To Understand the Rationale/Reasoning Generated/Drawn by the Teacher Model and also the Labels

!pip install -U 'accelerate==0.26.0'

import accelerate

from transformers import DataCollatorWithPadding

accelerate.__version__

training_arguments = TrainingArguments(

                                       output_dir = "./results",

                                       eval_strategy = "epoch",

                                       learning_rate = 5e-5,

                                       per_device_train_batch_size = 8,

                                       per_device_eval_batch_size = 8,

                                       num_train_epochs = 3,

                                       weight_decay = 0.01,

                                       save_strategy = "epoch",

                                       push_to_hub = False,

)

class MultiTaskTrainer(Trainer):

  def compute_loss(self, model, inputs, return_outputs = False, num_items_in_batch = None, **kwargs):

    labels = inputs.pop("labels")

    reasoning_ids = inputs.pop("reasoning_ids", None)

    outputs = model(**inputs)

    loss_function = torch.nn.CrossEntropyLoss()

    label_loss = loss_function(outputs.logits, labels)

    if reasoning_ids is not None:

      reasoning_outputs = model(input_ids = reasoning_ids, attention_mask = inputs["attention_mask"])

      reasoning_loss = loss_function(reasoning_outputs.logits, reasoning_ids)

      loss = label_loss + 0.5 * reasoning_loss

    else:

      loss = label_loss


    return (loss, outputs) if return_outputs else loss


student_model_trainer = MultiTaskTrainer(

                                         model = student_llm_model,

                                         args = training_arguments,

                                         train_dataset = processed_dataset,

                                         eval_dataset = processed_dataset,

                                         data_collator = DataCollatorWithPadding(tokenizer = llm_tokenizer)
)

student_model_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: faiazrex8 (faiazrex8-scaler) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.530785
2,No log,0.419531
3,No log,0.347946


TrainOutput(global_step=6, training_loss=0.5225234429041544, metrics={'train_runtime': 77.9763, 'train_samples_per_second': 0.385, 'train_steps_per_second': 0.077, 'total_flos': 398777693280.0, 'train_loss': 0.5225234429041544, 'epoch': 3.0})

In [21]:
print(processed_dataset)

Dataset({
    features: ['idx', 'sentence', 'label', 'input_ids', 'attention_mask', 'labels', 'reasoning_ids ', 'reasoning_mask'],
    num_rows: 10
})


In [22]:
student_model_save_path = "./result"

student_model_trainer.save_model(student_model_save_path)

llm_tokenizer.save_pretrained(student_model_save_path)

print("Distillation Complete! Student Model and Tokenzier Saved!")

Distillation Complete! Student Model and Tokenzier Saved!


In [23]:
#Evaluating the Student Model

from transformers import AutoModelForSequenceClassification

from transformers import AutoTokenizer

import torch

student_model = AutoModelForSequenceClassification.from_pretrained(student_model_save_path)

llm_model_tokenizer = AutoTokenizer.from_pretrained(student_model_save_path)

student_model.eval()

def predict_sentiment(input_text):

    input_tokens = llm_model_tokenizer(input_text, return_tensors = "pt", padding = True, truncation = True, max_length = 512)

    with torch.no_grad():

      output_tokens = student_model(**input_tokens)

      logits = output_tokens.logits

      predicted_sentiment_class = torch.argmax(logits, dim = -1).item()

    return predicted_sentiment_class


#Testing the Student Model

user_input_text = "are more deeply thought through than in most right thinking films"

sentiment_prediction = predict_sentiment(user_input_text)

print("Predicted sentiment class", sentiment_prediction)

Predicted sentiment class 1


In [24]:
#Giving new input and testing

model_test_prompt = "Predict the sentiment for this input - I am very happy today!"

model_response = teacher_model.invoke(model_test_prompt)

print(model_response.content)

<think>
Okay, so I need to figure out the sentiment of the sentence "I am very happy today!" Hmm, let's break this down. First, the word "happy" is a strong indicator of positive emotion. The use of "very" emphasizes the intensity of the happiness, which makes the sentiment even more positive. There are no negative words here, so it's not like a mixed sentiment. The exclamation mark at the end also adds to the enthusiasm, showing that the speaker is really feeling upbeat. I don't see any negative terms or context that might suggest otherwise. So, putting it all together, the sentiment is definitely positive.
</think>

The sentiment of the input "I am very happy today!" is **positive**. The use of the word "happy" and the emphasis with "very" clearly indicate a positive emotional state.


In [31]:
!pip install transformers --upgrade

from transformers import AutoModelForCausalLM

from transformers import AutoTokenizer

import torch


student_llm_model = AutoModelForCausalLM.from_pretrained(student_model_save_path)

student_llm_model_tokenizer = AutoTokenizer.from_pretrained(student_model_save_path)

student_llm_model.eval()


def generate_model_response(input_prompt, max_length = 100):

  input_tokens = student_llm_model_tokenizer(input_prompt, return_tensors = "pt", padding = True, truncation = True)

  with torch.no_grad():

    output_tokens = student_llm_model.generate(**input_tokens, max_length = max_length)

  student_llm_model_response = student_llm_model_tokenizer.decode(output_tokens[0], skip_special_tokens = True)

  return student_llm_model_response

test_prompt = "I am very joyful person!"

student_llm_model_response = generate_model_response(test_prompt)

print(student_llm_model_response)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at ./result and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


i am very joyful person!ighalitysibleersonٹduk guerrarandtinorandrrowuncetureital stressesʊbangrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowrrowumbliprrowrrowrrowrrowrrow


In [34]:
import pandas as pd

data_points = {

        "text": [],

        "reasoning" : [],

        "target" : []
}

for example in dataset:

  input_text = example["sentence"]

  reasoning = generate_model_reasoning(input_text)

  label = example["label"]

  data_points["text"].append(input_text)

  data_points["reasoning"].append(reasoning)

  data_points["target"].append(label)


df = pd.DataFrame(data_points)

df.to_csv("train.csv", index = False)